In [16]:
def calculate_buy_price(mid_price, spread):
    return mid_price + spread

def calculate_sell_price(mid_price, spread):
    return mid_price - spread

In [1]:
def active_strategy(dfY, total_capital, shares_capital, investment_pct, commission=0.002, buy_threshold=0.01, sell_threshold=0.01, stop_loss_threshold=0.05):
    shares = 0
    capital_history = []
    number_of_trades = 0
    total_comissions = 0
    portfolio_history = []
    investment_amt_sell_history = []
    investment_amt_buy_history = []

    no_trade_mode = False
    
    def buy_shares(shares_to_buy, buy_price, init=False):
        nonlocal total_capital, shares, number_of_trades, total_comissions

        commission_local = commission if init == False else 0
        
        total_capital -= shares_to_buy * buy_price * (1 + commission_local)
        shares += shares_to_buy
        
        number_of_trades += 1 if init is False else 0
        total_comissions += (shares_to_buy * buy_price * commission_local)
        
        portfolio_history.append((total_capital + shares * buy_price, 'Buy'))  # Append a tuple with portfolio value and action

    def sell_shares(shares_to_sell, sell_price, stop_loss=False):
        nonlocal total_capital, shares, number_of_trades, total_comissions
        total_capital += shares_to_sell * sell_price * (1 - commission)
        shares -= shares_to_sell
        
        number_of_trades += 1
        total_comissions += shares_to_sell * sell_price * commission

        message = 'Sell' if stop_loss is False else 'Stop Loss Sell'

        portfolio_history.append((total_capital + shares * sell_price, message))  # Append a tuple with portfolio value and action

    def no_transaction(sell_price, action):
        #portfolio_history.append((portfolio_history[-1][0], ''))  # Append a tuple with the same portfolio value and an empty action
        portfolio_history.append((total_capital + shares * sell_price, action))

        
    if shares_capital > 0:
        current_spread    =    dfY.iloc[0]['spread'] / 2
        current_buy_price =    calculate_buy_price(dfY.iloc[0]['mid_close'], current_spread)
        
        shares_to_buy = shares_capital / (current_buy_price * (1 + commission))
        buy_shares(shares_to_buy, current_buy_price, init=True)
    else:
        portfolio_history.append((total_capital, 'init no stocks'))

    # init
    investment_sell = shares_capital * investment_pct
    investment_buy = total_capital * investment_pct

    
    investment_amt_sell_history.append(investment_sell)
    investment_amt_buy_history.append(investment_buy)

    capital_history.append(total_capital)



    for i in range(1, len(dfY)):  # Start from the second row
        current_spread          =    dfY.iloc[i-1]['spread'] / 2  # Half of the spread to calculate bid and ask
        current_buy_price       =    calculate_buy_price(dfY.iloc[i-1]['mid_close'], current_spread)
        current_sell_price      =    calculate_sell_price(dfY.iloc[i-1]['mid_close'], current_spread)
        
        predicted_buy_price     =    calculate_buy_price(dfY.iloc[i]['prediction'], current_spread) # Using current spread, as we predict only the mid price
        predicted_sell_price    =    calculate_sell_price(dfY.iloc[i]['prediction'], current_spread)

        # if no_trade_mode == True and ((total_capital + shares * current_sell_price) >= (1 - stop_loss_threshold) * portfolio_history[0][0]):
        #     no_trade_mode = False
        
            # Check for stop loss condition within the main if statement
        if ((total_capital + shares * current_sell_price) <= (1 - stop_loss_threshold) * portfolio_history[0][0]) and (shares > 0):
            shares_to_sell = shares
            sell_shares(shares_to_sell, current_sell_price, stop_loss=True)
            no_trade_mode=True

        elif predicted_buy_price > (current_buy_price * (1 + buy_threshold) * (1 + commission)) and no_trade_mode == False:
            shares_to_buy = investment_buy / (current_buy_price * (1 + commission)) # Ammount of shares which can be traded discounted for comissions
            if total_capital >= investment_buy:
                buy_shares(shares_to_buy, current_buy_price)
            else:
                no_transaction(current_sell_price, 'Insufficient cash')

        elif predicted_sell_price < (current_sell_price * (1 - sell_threshold) * (1 - commission)) and no_trade_mode == False:
            # Sell shares
            shares_to_sell = investment_sell / (current_sell_price - (1 - commission)) # Ammount of shares which can be traded discounted for comissions
            if shares >= shares_to_sell:
                sell_shares(shares_to_sell, current_sell_price)
            else:
                no_transaction(current_sell_price, 'Insufficient shares')
                
        else:
            # No transaction
            message = 'Uncertain' if no_trade_mode == False else 'Stop Loss Mode'
            
            no_transaction(current_sell_price, message)

        capital_history.append(total_capital)
        
        investment_amt_sell_history.append(investment_sell)
        investment_amt_buy_history.append(investment_buy)
        
        investment_buy = capital_history[-1] * investment_pct
        investment_sell = (portfolio_history[-1][0] - capital_history[-1]) * investment_pct

    portfolio_df = pd.DataFrame(portfolio_history, columns=['Portfolio Value', 'Action'], index=dfY.index)
    capital_df = pd.DataFrame(capital_history, columns=['Available Cash'], index=dfY.index)
    investment_amt_sell_df   = pd.DataFrame(investment_amt_sell_history, columns=['Investment Ammount'], index=dfY.index)
    investment_amt_buy_df    = pd.DataFrame(investment_amt_buy_history, columns=['Investment Ammount'], index=dfY.index)




    print(f'Number of executed trades: {number_of_trades}')
    print(f'Total commissions paid: {total_comissions}')

    
    return (portfolio_df, capital_df, investment_amt_sell_df, investment_amt_buy_df, number_of_trades, total_comissions)


In [2]:
def passive_strategy(dfY, capital, shares, commission, n, stop_loss_threshold=0.05):
    portfolio_history = [capital]
    capital_history = [(capital, 'init')]
    capital_per_trade = capital / len(dfY) * n
    number_of_trades = 0

    stop_loss = False
    no_trade_mode = False

    for i in range(1, len(dfY)):
        current_spread = dfY.iloc[i - 1]['spread'] / 2
        current_mid_price = dfY.iloc[i - 1]['mid_close']
        current_buy_price = calculate_buy_price(current_mid_price, current_spread)
        current_sell_price = calculate_sell_price(current_mid_price, current_spread)
        
         # Check for stop loss condition
        if ((capital + shares * current_sell_price) <= (1 - stop_loss_threshold) * capital_history[0][0]) and (shares > 0):
            stop_loss = True
            no_trade_mode = True
            
            shares_to_sell = shares
            capital += shares_to_sell * current_sell_price * (1 - commission)
            shares = 0
            number_of_trades += 1

        elif i == 1 or (i - 1) % n == 0 and no_trade_mode == False:
            # Buy shares on the very first step and every N steps
            shares_to_buy = capital_per_trade / (current_buy_price * (1 + commission))
            capital -= shares_to_buy * current_buy_price * (1 + commission)
            shares += shares_to_buy
            number_of_trades += 1


        # Calculate portfolio value at each step
        portfolio_value = capital + shares * current_sell_price
        portfolio_history.append(portfolio_value)

        if stop_loss:
            message = 'Stop Loss Sell'
        elif no_trade_mode:
            message = 'Stop Loss Mode'
        else:
            message = ''
        
        capital_history.append((capital, message))
        stop_loss = False

    portfolio_df = pd.DataFrame(portfolio_history, columns=['Portfolio Value'], index=dfY.index)
    capital_df = pd.DataFrame(capital_history, columns=['Available Cash', 'Action'], index=dfY.index)
    
    print(f'number of executed trades: {number_of_trades}')
    return portfolio_df, capital_df
